In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os
import pdb

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = '' # probably not needed
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    # transforming deviation to the matrix form
    devs = heats_data.loc['deviation'].to_numpy()
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs)))
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC' or 'overfilled':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    if _calor_setup_type == 'ampoule':
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    vol_active = 0.988 # terrible solution. Gotta introduce new variable for operating volume 
    vol_init = 15      # of calorimetric cell for overfilled mode
    dif_conc = []        
             
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            
            if _calor_setup_type == 'overfilled':
                factor = 1 - (volumes[int(np.sum(ser_counts[:s])) + k - s] / vol_active)
            else:
                factor = 1
            
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) +
                                                                                       k] * factor)
            
    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)
    
    dif_vol = np.diff(volumes)
    dif_vol = np.array(np.hstack((volumes[0] - vol_init, dif_vol)))
    
    # finding vector of experimental values
    
    q_corr = deepcopy(heats)
    
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            
            if _calor_setup_type == 'overfilled':
                q_corr[i] -= vol_active * dif_conc[i, dH_ind[j]] * dH_par[j] 
            else:
                q_corr[i] -= (volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j] +
                              dif_vol[i] * np.array(c_res_out)[i, dH_ind[j]] * dH_par[j])
    
    if _calor_setup_type == 'overfilled' or _calor_setup_type == 'DSC':
        c_eq = []
        for s in range(ser_num):
            for k in range(ser_counts[s] - 1):
                c_eq.append(c_res_out[int(np.sum(ser_counts[:s])) + k])
    else:
        c_eq = dif_conc
    
    if _calor_setup_type == 'overfilled':
        A = (vol_active * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff)
    
    else:
        A = ((np.array(volumes).reshape((len(volumes), 1)) * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) +
             dif_vol.reshape((len(dif_vol), 1)) * np.array(c_eq)[:, dH_ind_wtf])
    
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    F_min, heats_calc = 0, []
    
    for i in range(len(heats)): 
        
        if _calor_setup_type == 'overfilled':
            F_min += (vol_active * np.sum(dH_res_out * dif_conc[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
            
            heats_calc.append((np.sum(dH_res_out * dif_conc[i] * vol_active * _calor_setup_type_coeff[i])))
        
        else:
            F_min += ((np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) +
                       np.sum(dH_res_out * c_eq[i] * dif_vol[i]) - heats[i]) ** 2)
            
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i]))
                              + np.sum(dH_res_out * c_eq[i] * dif_vol[i]))
    print('\neq con')
    print(np.array(c_res_out))
    print('\ndif con')
    print(dif_conc)
    print('\nvol')
    print(volumes)
    print('\ndH par')
    print(dH_par)
    print('\nsum of terms')
    print(np.array(heats_calc))
    print(heats)
    
    #print(sum(heats))
    #print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    
    # calculating st. dev. of dH_res / checking the quality of linear regression
    # consider making it in separate function
    
    disp = (F_min / (len(heats) - len(dH_ind_wtf)) * (np.diag(LTP))) ** 0.5  
    
    return F_min, dH_res_out, disp

In [4]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

from copy import deepcopy
import eq_evaluator
# import also script cl_enthalpies when needed

# main Hooke-Jeeves procedure --------------------------------
def cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)
    print(new_min)

    for i in range(max_iter):
        print('\niteration', i, lg_k[tar_num])
        if F_min <= new_min:
        
            if step_init < HJ_eps:
                print("p1 " + str(step_init) + " " + str(HJ_eps))
                break
            
            else: 
                 
                step_init /= 2
            
                basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
                print('\nF min more ', new_min, F_min, lg_k[tar_num])
                print(str(step_init))
        else:
            F_min = deepcopy(new_min)
            print('\nF min less', new_min, F_min, lg_k[tar_num])
            print(str(step_init))
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('\nF min less after', new_min, F_min, lg_k[tar_num])
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
                print('\nconstant less after', basis_point[tar_num], lg_k[tar_num]) 
            else:
                F_min = new_min

            basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
            if F_min <= new_min:
            
                print('\nF min p2', new_min, F_min, lg_k[tar_num])
                if step_init < HJ_eps:
                    print("p2")
                    break
            
                else: 
                
                    step_init /= 2
        
            else:

                F_min = deepcopy(new_min)
            
                print('\nF min p3.1', new_min, F_min, lg_k[tar_num])
            
                basis_point_n = deepcopy(lg_k)
                lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
            
                c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                        con_matrix, ign_indices)
    
                new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
                if F_min <= new_min:
                    lg_k = deepcopy(basis_point_n)
            
                print('\nF min p3.2', new_min, F_min, lg_k[tar_num])
    
    return F_min, lg_k, dH_res_out, c_res_out, disp

# exploratory move function used in main function ------------------------
def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)

    #if F_min > new_min:
    F_min = new_min
    
    for j in range(len(tar_num)): 
        #pdb.set_trace()
        print('input', lg_k[tar_num[j]], F_min)
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        #print('\na1 ', new_min, F_min)
        new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                   _calor_setup_type_coeff, _calor_setup_type)
        print('right', lg_k[tar_num[j]], new_min, F_min)
        #print('\na2 ', new_min, F_min)
        if F_min <= new_min:
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                  , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('left', lg_k[tar_num[j]], new_min, F_min)
            #print('\na3 ', new_min, F_min)
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                #print('\na4 ', new_min, F_min)
            #else:
                
                #new_min = deepcopy(F_min)
                
        #else:
            
            #new_min = deepcopy(F_min)
    print('return', basis_point[tar_num[j]], lg_k[tar_num[j]])
    #print('\na5 ', new_min, F_min)
    return basis_point, lg_k, new_min, c_res_out, disp

In [5]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------
import numpy as np
import eq_evaluator
# import also script cl_enthalpies when needed

def cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                      ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                      _calor_setup_type_coeff, _calor_setup_type):
    
    # creating constants vectors ---------------------------------
    lg_k_1, lg_k_2 = np.array(lg_k), np.array(lg_k)
    heats_one, heats_two = [0] * len(heats), [0] * len(heats)
    jc_mtrx = [[0] * len(heats) for i in range(len(tar_num))]

    for i in range(len(tar_num)):
        # calculating the heats for Jacobi matrix computing
        lg_k_1[tar_num[i]] += HJ_eps
        lg_k_2[tar_num[i]] -= HJ_eps
    
        # if lg_k is overestimated
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_1,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])
    
        for o in range(len(heats)):
            heats_one[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # if lg_k is underestimated    
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_2,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])    
   
        for o in range(len(heats)):
            heats_two[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # calculating the Jacobi matrix    
        jc_mtrx[i] = (np.array(heats_one) - np.array(heats_two)) / (2 * HJ_eps)

        jc_mtrx, devs = np.array(jc_mtrx).astype(np.float32), devs.astype(np.float32)

    cov_matrix = (F_min / (len(heats) - len(tar_num))) * np.linalg.inv(np.dot(np.dot(jc_mtrx, devs), np.transpose(jc_mtrx)))

    errs_values = np.diag(cov_matrix) ** 0.5
    return errs_values

In [6]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_6.xlsx"
_calor_setup_type = 'DSC' # use 'DSC' for test_1, test_2 and test_5; use 'ampoule' for test_3; use 'overfilled' for test_4

max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.0001, 0.5

# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)

# calculation of eq concentration (starting point)
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

# calculation of F_min and unknown enthalpies (starting point)
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# F_min assignment for further calculations
new_min = F_min

# constant refinment
if len(tar_num) > 0:
    F_min, lg_k, dH_res_out, c_res_out, disp = cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)


F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# error estimation
if len(tar_num) > 0:
    errs_values = cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                                    ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                                    _calor_setup_type_coeff, _calor_setup_type)


eq con
[[9.09090909e-17 1.00000000e-03 9.09090909e-18 8.26446281e-33]
 [4.80628320e-04 3.80937585e-03 1.83089392e-04 8.79979466e-07]
 [9.69774435e-04 3.63115724e-03 3.52140346e-04 3.41496705e-06]
 [1.46675059e-03 3.46449370e-03 5.08154817e-04 7.45336377e-06]
 [1.97089999e-03 3.30857422e-03 6.52086891e-04 1.28519805e-05]
 [2.48160007e-03 3.16263100e-03 7.84838531e-04 1.94765535e-05]
 [2.99826431e-03 3.02594153e-03 9.07257248e-04 2.72019703e-05]
 [3.52034305e-03 2.89782936e-03 1.02013535e-03 3.59122637e-05]
 [4.04732338e-03 2.77766384e-03 1.12421038e-03 4.55004297e-05]
 [4.57872839e-03 2.66485901e-03 1.22016656e-03 5.58681127e-05]
 [5.11411588e-03 2.55887199e-03 1.30863678e-03 6.69252015e-05]
 [5.65307684e-03 2.45920088e-03 1.39020515e-03 7.85893657e-05]
 [6.19523366e-03 2.36538255e-03 1.46540976e-03 9.07855586e-05]
 [6.74023816e-03 2.27699017e-03 1.53474560e-03 1.03445509e-04]
 [7.28776969e-03 2.19363076e-03 1.59866757e-03 1.16507211e-04]
 [7.83753319e-03 2.11494274e-03 1.65759339e-03 


eq con
[[2.40253075e-17 1.00000000e-03 7.59746932e-17 1.82531537e-32]
 [5.67983087e-05 3.38501119e-03 6.07988707e-04 3.45327303e-07]
 [1.34757533e-04 2.79433003e-03 1.19077786e-03 1.60466286e-06]
 [2.45134832e-04 2.23968052e-03 1.73616541e-03 4.25594616e-06]
 [4.04833690e-04 1.73866687e-03 2.22583529e-03 9.01093115e-06]
 [6.34963869e-04 1.31326186e-03 2.63694060e-03 1.67436201e-05]
 [9.51960199e-04 9.80535085e-04 2.95176602e-03 2.80996377e-05]
 [1.35532173e-03 7.39873378e-04 3.17102599e-03 4.29776042e-05]
 [1.82785344e-03 5.73260267e-04 3.31354759e-03 6.05667938e-05]
 [2.34809784e-03 4.58259889e-04 3.40273427e-03 7.98995300e-05]
 [2.89904703e-03 3.77101173e-04 3.45710957e-03 1.00223232e-04]
 [3.46942573e-03 3.18006660e-04 3.48894248e-03 1.21046268e-04]
 [4.05216955e-03 2.73598451e-04 3.50591387e-03 1.42065574e-04]
 [4.64285855e-03 2.39260552e-04 3.51282532e-03 1.63095511e-04]
 [5.23866891e-03 2.12043622e-04 3.51274130e-03 1.84020886e-04]
 [5.83774486e-03 1.90008897e-04 3.50767257e-03 


eq con
[[2.40253075e-17 1.00000000e-03 7.59746932e-17 5.77215400e-32]
 [5.66623035e-05 3.38561685e-03 6.06641383e-04 1.08699178e-06]
 [1.33951124e-04 2.79693748e-03 1.18475656e-03 5.01851796e-06]
 [2.42341958e-04 2.24582133e-03 1.72109093e-03 1.31896244e-05]
 [3.97037935e-04 1.74943817e-03 2.19649693e-03 2.75779897e-05]
 [6.16229026e-04 1.32822103e-03 2.58828743e-03 5.04376279e-05]
 [9.13426481e-04 9.97094066e-04 2.88011434e-03 8.31923376e-05]
 [1.28796081e-03 7.54733259e-04 3.07394531e-03 1.25198402e-04]
 [1.72555921e-03 5.84410724e-04 3.18895245e-03 1.74011484e-04]
 [2.20814090e-03 4.65257850e-04 3.24878141e-03 2.26854434e-04]
 [2.72092543e-03 3.80334588e-04 3.27252116e-03 2.81578248e-04]
 [3.25378818e-03 3.18105372e-04 3.27310758e-03 3.36782531e-04]
 [3.80019476e-03 2.71175198e-04 3.25878584e-03 3.91617125e-04]
 [4.35594724e-03 2.34833724e-04 3.23476753e-03 4.45579997e-04]
 [4.91829644e-03 2.06030413e-04 3.20439492e-03 4.98380150e-04]
 [5.48539049e-03 1.82739383e-04 3.16985724e-03 


eq con
[[2.40253075e-17 1.00000000e-03 7.59746932e-17 3.24592073e-32]
 [5.67492061e-05 3.38522983e-03 6.07502331e-04 6.13066716e-07]
 [1.34465074e-04 2.79527504e-03 1.18859539e-03 2.84212736e-06]
 [2.44115206e-04 2.24191788e-03 1.73067107e-03 7.51293040e-06]
 [4.01961267e-04 1.74261677e-03 2.21506306e-03 1.58332585e-05]
 [6.27983166e-04 1.31878375e-03 2.61891612e-03 2.92462098e-05]
 [9.37442777e-04 9.86678177e-04 2.92496241e-03 4.87601527e-05]
 [1.32971326e-03 7.45402421e-04 3.13435943e-03 7.41151166e-05]
 [1.78869090e-03 5.77417821e-04 3.26606994e-03 1.03886897e-04]
 [2.29420264e-03 4.60875449e-04 3.34360795e-03 1.36410288e-04]
 [2.83008942e-03 3.78313485e-04 3.38572734e-03 1.70393153e-04]
 [3.38551711e-03 3.18043810e-04 3.40495958e-03 2.04992029e-04]
 [3.95362956e-03 2.72682490e-04 3.40920585e-03 2.39689605e-04]
 [4.53010010e-03 2.37579474e-04 3.40342915e-03 2.74172892e-04]
 [5.11214320e-03 2.09748932e-04 3.39080464e-03 3.08251912e-04]
 [5.69791471e-03 1.87220877e-04 3.37341847e-03 

eq con
[[2.40253075e-17 1.00000000e-03 7.59746932e-17 3.74833078e-32]
 [5.67318809e-05 3.38530698e-03 6.07330705e-04 7.07542319e-07]
 [1.34362241e-04 2.79560749e-03 1.18782766e-03 3.27740922e-06]
 [2.43758530e-04 2.24270176e-03 1.72874663e-03 8.65348744e-06]
 [4.00963713e-04 1.74399366e-03 2.21131174e-03 1.82076980e-05]
 [6.25580443e-04 1.32069853e-03 2.61268384e-03 3.35637147e-05]
 [9.32490676e-04 9.88799744e-04 2.91576718e-03 5.58338199e-05]
 [1.32104300e-03 7.47307237e-04 3.12187955e-03 8.46901880e-05]
 [1.77550973e-03 5.78847571e-04 3.25002927e-03 1.18497811e-04]
 [2.27615327e-03 4.61773060e-04 3.32376335e-03 1.55357276e-04]
 [2.80710178e-03 3.78728424e-04 3.36190982e-03 1.93795737e-04]
 [3.35766970e-03 3.18056485e-04 3.37708682e-03 2.32852121e-04]
 [3.92106963e-03 2.72371121e-04 3.37726869e-03 2.71938168e-04]
 [4.49300412e-03 2.37010254e-04 3.36747169e-03 3.10699654e-04]
 [5.07069791e-03 2.08975142e-04 3.35090693e-03 3.48923407e-04]
 [5.65230802e-03 1.86284711e-04 3.32968411e-03 3

...


eq con
[[2.39356377e-17 1.00000000e-03 7.60643630e-17 3.86103098e-32]
 [5.64773411e-05 3.38507256e-03 6.07545005e-04 7.27662087e-07]
 [1.33772105e-04 2.79511082e-03 1.18823085e-03 3.37088111e-06]
 [2.42723363e-04 2.24191445e-03 1.72928609e-03 8.90134182e-06]
 [3.99348260e-04 1.74290276e-03 2.21187808e-03 1.87322537e-05]
 [6.23245643e-04 1.31933726e-03 2.61307159e-03 3.45372385e-05]
 [9.29327438e-04 9.87265201e-04 2.91567303e-03 5.74625150e-05]
 [1.31698093e-03 7.45721324e-04 3.12098931e-03 8.71663417e-05]
 [1.77050094e-03 5.77297578e-04 3.24812047e-03 1.21956609e-04]
 [2.27016543e-03 4.60298142e-04 3.32072535e-03 1.59870198e-04]
 [2.80011986e-03 3.77337735e-04 3.35770928e-03 1.99386966e-04]
 [3.34969516e-03 3.16746341e-04 3.37173258e-03 2.39516512e-04]
 [3.91211678e-03 2.71133832e-04 3.37079043e-03 2.79653728e-04]
 [4.48309614e-03 2.35837692e-04 3.35990887e-03 3.19435075e-04]
 [5.05986366e-03 2.07859934e-04 3.34230310e-03 3.58642453e-04]
 [5.64057974e-03 1.85220530e-04 3.32008419e-03 


eq con
[[2.39356377e-17 1.00000000e-03 7.60643630e-17 3.86320208e-32]
 [5.64772667e-05 3.38507289e-03 6.07544265e-04 7.28069415e-07]
 [1.33771664e-04 2.79511226e-03 1.18822754e-03 3.37275608e-06]
 [2.42721834e-04 2.24191783e-03 1.72927780e-03 8.90624840e-06]
 [3.99343991e-04 1.74290869e-03 2.21186196e-03 1.87424501e-05]
 [6.23235378e-04 1.31934549e-03 2.61304486e-03 3.45557366e-05]
 [9.29306321e-04 9.87274313e-04 2.91563369e-03 5.74927446e-05]
 [1.31694403e-03 7.45729498e-04 3.12093605e-03 8.72114240e-05]
 [1.77044492e-03 5.77303709e-04 3.24805218e-03 1.22018761e-04]
 [2.27008883e-03 4.60301989e-04 3.32064106e-03 1.59950638e-04]
 [2.80002246e-03 3.77339515e-04 3.35760832e-03 1.99486146e-04]
 [3.34957735e-03 3.16746402e-04 3.37161465e-03 2.39634385e-04]
 [3.91197924e-03 2.71132513e-04 3.37065553e-03 2.79789946e-04]
 [4.48293968e-03 2.35835279e-04 3.35975724e-03 3.19589120e-04]
 [5.05968913e-03 2.07856657e-04 3.34213512e-03 3.58813710e-04]
 [5.64038797e-03 1.85216571e-04 3.31990034e-03 


eq con
[[2.39356377e-17 1.00000000e-03 7.60643630e-17 3.86428809e-32]
 [5.64772295e-05 3.38507306e-03 6.07543894e-04 7.28273164e-07]
 [1.33771443e-04 2.79511297e-03 1.18822588e-03 3.37369396e-06]
 [2.42721070e-04 2.24191952e-03 1.72927366e-03 8.90870268e-06]
 [3.99341856e-04 1.74291165e-03 2.21185389e-03 1.87475503e-05]
 [6.23230243e-04 1.31934961e-03 2.61303149e-03 3.45649891e-05]
 [9.29295758e-04 9.87278871e-04 2.91561401e-03 5.75078650e-05]
 [1.31692557e-03 7.45733587e-04 3.12090941e-03 8.72339732e-05]
 [1.77041690e-03 5.77306776e-04 3.24801803e-03 1.22049847e-04]
 [2.27005052e-03 4.60303913e-04 3.32059891e-03 1.59990872e-04]
 [2.79997375e-03 3.77340405e-04 3.35755782e-03 1.99535752e-04]
 [3.34951843e-03 3.16746433e-04 3.37155566e-03 2.39693340e-04]
 [3.91191045e-03 2.71131853e-04 3.37058806e-03 2.79858076e-04]
 [4.48286143e-03 2.35834072e-04 3.35968140e-03 3.19666165e-04]
 [5.05960183e-03 2.07855018e-04 3.34205110e-03 3.58899364e-04]
 [5.64029206e-03 1.85214590e-04 3.31980839e-03 


eq con
[[2.39356377e-17 1.00000000e-03 7.60643630e-17 3.86591768e-32]
 [5.64771737e-05 3.38507331e-03 6.07543339e-04 7.28578894e-07]
 [1.33771112e-04 2.79511405e-03 1.18822340e-03 3.37510125e-06]
 [2.42719923e-04 2.24192205e-03 1.72926744e-03 8.91238535e-06]
 [3.99338651e-04 1.74291610e-03 2.21184179e-03 1.87552032e-05]
 [6.23222539e-04 1.31935579e-03 2.61301142e-03 3.45788724e-05]
 [9.29279910e-04 9.87285711e-04 2.91558448e-03 5.75305526e-05]
 [1.31689787e-03 7.45739722e-04 3.12086944e-03 8.72678070e-05]
 [1.77037486e-03 5.77311377e-04 3.24796679e-03 1.22096490e-04]
 [2.26999304e-03 4.60306800e-04 3.32053565e-03 1.60051239e-04]
 [2.79990065e-03 3.77341741e-04 3.35748206e-03 1.99610181e-04]
 [3.34943002e-03 3.16746479e-04 3.37146716e-03 2.39781796e-04]
 [3.91180724e-03 2.71130864e-04 3.37048683e-03 2.79960298e-04]
 [4.48274402e-03 2.35832261e-04 3.35956761e-03 3.19781763e-04]
 [5.05947086e-03 2.07852559e-04 3.34192505e-03 3.59027877e-04]
 [5.64014816e-03 1.85211619e-04 3.31967043e-03 


eq con
[[2.39356377e-17 1.00000000e-03 7.60643630e-17 3.86700446e-32]
 [5.64771365e-05 3.38507347e-03 6.07542968e-04 7.28782784e-07]
 [1.33770891e-04 2.79511477e-03 1.18822174e-03 3.37603977e-06]
 [2.42719157e-04 2.24192374e-03 1.72926329e-03 8.91484130e-06]
 [3.99336515e-04 1.74291907e-03 2.21183372e-03 1.87603067e-05]
 [6.23217402e-04 1.31935991e-03 2.61299804e-03 3.45881309e-05]
 [9.29269341e-04 9.87290271e-04 2.91556479e-03 5.75456823e-05]
 [1.31687940e-03 7.45743813e-04 3.12084279e-03 8.72903695e-05]
 [1.77034682e-03 5.77314445e-04 3.24793261e-03 1.22127595e-04]
 [2.26995471e-03 4.60308725e-04 3.32049347e-03 1.60091495e-04]
 [2.79985191e-03 3.77342631e-04 3.35743153e-03 1.99659815e-04]
 [3.34937106e-03 3.16746509e-04 3.37140814e-03 2.39840783e-04]
 [3.91173842e-03 2.71130204e-04 3.37041932e-03 2.80028464e-04]
 [4.48266573e-03 2.35831054e-04 3.35949174e-03 3.19858848e-04]
 [5.05938352e-03 2.07850919e-04 3.34184099e-03 3.59113574e-04]
 [5.64005220e-03 1.85209637e-04 3.31957844e-03 

In [7]:
print(np.array(c_res_out))
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[[2.39381965e-17 1.00000000e-03 7.60618042e-17 3.86837491e-32]
 [5.64842374e-05 3.38508086e-03 6.07535293e-04 7.29070102e-07]
 [1.33786787e-04 2.79513196e-03 1.18820325e-03 3.37733756e-06]
 [2.42745449e-04 2.24195334e-03 1.72923040e-03 8.91814369e-06]
 [3.99373570e-04 1.74296271e-03 2.21178349e-03 1.87668956e-05]
 [6.23262314e-04 1.31941614e-03 2.61293049e-03 3.45994529e-05]
 [9.29314965e-04 9.87353310e-04 2.91548434e-03 5.75630970e-05]
 [1.31691730e-03 7.45806333e-04 3.12075565e-03 8.73149837e-05]
 [1.77037135e-03 5.77371619e-04 3.24784279e-03 1.22160241e-04]
 [2.26996369e-03 4.60358925e-04 3.32040204e-03 1.60132723e-04]
 [2.79984524e-03 3.77386054e-04 3.35733802e-03 1.99709904e-04]
 [3.34934953e-03 3.16783996e-04 3.37131164e-03 2.39899797e-04]
 [3.91170305e-03 2.71162690e-04 3.37031899e-03 2.80096314e-04]
 [4.48261755e-03 2.35859375e-04 3.35938692e-03 3.19935345e-04]
 [5.05932348e-03 2.07875772e-04 3.34173124e-03 3.59198468e-04]
 [5.63998115e-03 1.85231587e-04 3.31946349e-03 3.977553

In [8]:
#print('\nStoich coeff data')
#print(st_coeff_data)

#print('\nlg K')
#print(lg_k_data)

#print('\nconcentrations')
#print(con_data)

#print('\ntype con')
#print(type_con)

#print('\nexperimental data')
#print(heats_data)

print('\n_calor_setup_type_coeff')
print(_calor_setup_type_coeff)

print('\ntargets')
print(targets)

#print('\ndelta_H')
#print(delta_H)


_calor_setup_type_coeff
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

targets
            0   1    2
0  constants   HL  H2L


In [9]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[21]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]
 [2 1]]

prod names
['H', 'L', 'HL', 'H2L']
['H', 'L']

const
[[0.        ]
 [0.        ]
 [3.50213623]
 [4.82928467]]

concentrations
[[1.00000000e-16 1.00000000e-03]
 [6.65477670e-04 3.99334522e-03]
 [1.32874472e-03 3.98671255e-03]
 [1.98981213e-03 3.98010188e-03]
 [2.64869085e-03 3.97351309e-03]
 [3.30539171e-03 3.96694608e-03]
 [3.95992550e-03 3.96040075e-03]
 [4.61230292e-03 3.95387697e-03]
 [5.26253463e-03 3.94737465e-03]
 [5.91063117e-03 3.94089369e-03]
 [6.55660306e-03 3.93443397e-03]
 [7.20046073e-03 3.92799539e-03]
 [7.84221453e-03 3.92157785e-03]
 [8.48187477e-03 3.91518125e-03]
 [9.11945166e-03 3.90880548e-03]
 [9.75495538e-03 3.90245045e-03]
 [1.03883960e-02 3.89611604e-03]
 [1.10197836e-02 3.88980216e-03]
 [1.16491281e-02 3.88350872e-03]
 [1.22764394e-02 3.87723561e-03]
 [1.29017274e-02 3.87098273e-03]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object



In [10]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[2.39381965e-17 1.00000000e-03 7.60618042e-17 3.86837491e-32]
 [5.64842374e-05 3.38508086e-03 6.07535293e-04 7.29070102e-07]
 [1.33786787e-04 2.79513196e-03 1.18820325e-03 3.37733756e-06]
 [2.42745449e-04 2.24195334e-03 1.72923040e-03 8.91814369e-06]
 [3.99373570e-04 1.74296271e-03 2.21178349e-03 1.87668956e-05]
 [6.23262314e-04 1.31941614e-03 2.61293049e-03 3.45994529e-05]
 [9.29314965e-04 9.87353310e-04 2.91548434e-03 5.75630970e-05]
 [1.31691730e-03 7.45806333e-04 3.12075565e-03 8.73149837e-05]
 [1.77037135e-03 5.77371619e-04 3.24784279e-03 1.22160241e-04]
 [2.26996369e-03 4.60358925e-04 3.32040204e-03 1.60132723e-04]
 [2.79984524e-03 3.77386054e-04 3.35733802e-03 1.99709904e-04]
 [3.34934953e-03 3.16783996e-04 3.37131164e-03 2.39899797e-04]
 [3.91170305e-03 2.71162690e-04 3.37031899e-03 2.80096314e-04]
 [4.48261755e-03 2.35859375e-04 3.35938692e-03 3.19935345e-04]
 [5.05932348e-03 2.07875772e-04 3.34173124e-03 3.59198468e-04]
 [5.63998115e-03 1.85231587e-04 3.31946349e-03

In [11]:
print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

print('\ncalculated error of enthalpy')
print(disp)

if len(tar_num) > 0:
    print('\ncalculated error of constant')
    print(errs_values)


lg K
[[0.        ]
 [0.        ]
 [3.50213623]
 [4.82928467]]

F_min
3.938043469325817e-09

Final enthalpies
[0.0 0.0 9.990968089318486 12.878265298386111]

calculated error of enthalpy
[0.00079711 0.00679208]

calculated error of constant
[0.00119167 0.00132428]


In [12]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

#lg_k is array

In [13]:
import matplotlib.pyplot as plt
from copy import copy, deepcopy

def risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type):

    s = [[0] * 42 for i in range(42)]
    const_x = [0] * 42
    const_y = [0] * 42
    s = np.array(s)

    i, a = 2.5, 0

    for counter_x in range(len(s)):
        j = 3.3
        const_x[counter_x] = round(i, 2)
        for counter_y in range(len(s)):
            lg_k[2], lg_k[3] = np.array(i), np.array(j)
            const_y[counter_y] = round(j, 2)
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                            con_matrix, ign_indices)
        
            F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
            
            s[counter_x, counter_y] = float(F_min * 100000000)
        
            j += 0.05
    
        i += 0.05

    s = pd.DataFrame(data = s, index = const_y, columns = const_x)

    fig, ax = plt.subplots(figsize=(20, 20))
    im = ax.imshow(s, cmap = 'RdYlGn_r')
    ax.set_xticks(np.arange(len(const_x)))
    ax.set_yticks(np.arange(len(const_y)))

    ax.set_xlabel('lg K1')
    ax.set_ylabel('lg K2')

    ax.set_xticklabels(const_x)
    ax.set_yticklabels(const_y)

    plt.show()
    return s

In [14]:
#comment this section for reducing the waiting time
#s = risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff, _calor_setup_type)
#with pd.ExcelWriter('d:/KEVreplica/output/calorimetry/test_6.xlsx', mode = "w") as output:
    #s.to_excel(output)

In [15]:
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type) 


eq con
[[2.39381965e-17 1.00000000e-03 7.60618042e-17 3.86837491e-32]
 [5.64842374e-05 3.38508086e-03 6.07535293e-04 7.29070102e-07]
 [1.33786787e-04 2.79513196e-03 1.18820325e-03 3.37733756e-06]
 [2.42745449e-04 2.24195334e-03 1.72923040e-03 8.91814369e-06]
 [3.99373570e-04 1.74296271e-03 2.21178349e-03 1.87668956e-05]
 [6.23262314e-04 1.31941614e-03 2.61293049e-03 3.45994529e-05]
 [9.29314965e-04 9.87353310e-04 2.91548434e-03 5.75630970e-05]
 [1.31691730e-03 7.45806333e-04 3.12075565e-03 8.73149837e-05]
 [1.77037135e-03 5.77371619e-04 3.24784279e-03 1.22160241e-04]
 [2.26996369e-03 4.60358925e-04 3.32040204e-03 1.60132723e-04]
 [2.79984524e-03 3.77386054e-04 3.35733802e-03 1.99709904e-04]
 [3.34934953e-03 3.16783996e-04 3.37131164e-03 2.39899797e-04]
 [3.91170305e-03 2.71162690e-04 3.37031899e-03 2.80096314e-04]
 [4.48261755e-03 2.35859375e-04 3.35938692e-03 3.19935345e-04]
 [5.05932348e-03 2.07875772e-04 3.34173124e-03 3.59198468e-04]
 [5.63998115e-03 1.85231587e-04 3.31946349e-03 